# YOLO Trainning Code
### Prerequisite Step
**This portion of our code deals with loading the Google Drive on to the local directory of Google Collab**

In [1]:
# Enable GPU in google collab if not on this portion will flag error.
!nvidia-smi

Fri Aug 12 08:16:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 Bank_OOP_HW.py  'Colab Notebooks'   Models.zip  'My Drive'   PlantVillage
 Classroom	  DF-GAN	     model.zip	  OOP_HW.py   yolov3


### 1) Clone the Darknet
**This Porition is essentially responsible for cloning Darknet because Yolo was developed using Darknet framework and runs only on that particular framework**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


### 2) Compile Darknet using Nvidia GPU
**This porition is responsible for Compiling Darknet using Nvidia GPU and making slight adjustments to make file**



In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1150

### 3) Configure Darknet network for training YOLO V3

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "mask" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/train.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-12 08:16:43--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  17.1MB/s    in 10s     

2022-08-12 08:16:54 (15.4 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]



### 4) Extract Images

**The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" if any other name used .zip file and folder name is adjustment accordingly on Google Drive**

In [10]:
!unzip /content/gdrive/MyDrive/yolov3/images.zip -d data/obj

Archive:  /content/gdrive/MyDrive/yolov3/images.zip
replace data/obj/data/1920x1080 Banner 2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [11]:
#This part is entirely optional
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/data/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.559259 0.403743 0.200000 0.155080

['0', '0.559259', '0.403743', '0.200000', '0.155080']
0 data/obj/data/images - 2022-07-31T105631.283.txt
0 0.559259 0.403743 0.200000 0.155080
0 0.504444 0.497778 0.653333 0.302222

['0', '0.504444', '0.497778', '0.653333', '0.302222']
1 data/obj/data/images (86).txt
0 0.504444 0.497778 0.653333 0.302222
0 0.521154 0.479381 0.650000 0.690722

['0', '0.521154', '0.479381', '0.650000', '0.690722']
2 data/obj/data/images - 2022-07-31T105629.455.txt
0 0.521154 0.479381 0.650000 0.690722
0 0.509091 0.513661 0.647273 0.622951

['0', '0.509091', '0.513661', '0.647273', '0.622951']
3 data/obj/data/images (22).txt
0 0.509091 0.513661 0.647273 0.622951
0 0.471111 0.473333 0.480000 0.351111

['0', '0.471111', '0.473333', '0.480000', '0.351111']
4 data/obj/data/images - 2022-07-31T105629.179.txt
0 0.471111 0.473333 0.480000 0.351111
0 0.500000 0.464758 0.882883 0.735683

['0', '0.500000', '0.464758', '0.882883', '0.735683']
5 data/obj/data/images - 2022-07-31

In [12]:
import glob
images_list = glob.glob("data/obj/data/*.jpg")
im = glob.glob("data/obj/data/*.png")
images_list = images_list+ im
print(images_list)

['data/obj/data/images (49).jpg', 'data/obj/data/medical_mask.width-320.jpg', 'data/obj/data/images (22).jpg', 'data/obj/data/images - 2022-07-31T105630.965.jpg', 'data/obj/data/images - 2022-07-31T105628.768.jpg', 'data/obj/data/images (53).jpg', 'data/obj/data/surgical_mask_still_life-1296x728-header2.jpg', 'data/obj/data/images - 2022-07-31T105630.964.jpg', 'data/obj/data/images (66).jpg', 'data/obj/data/images - 2022-07-31T105629.084.jpg', 'data/obj/data/download (7).jpg', 'data/obj/data/PPE-100_1-650x650.jpg', 'data/obj/data/images (92).jpg', 'data/obj/data/MasksSaveLives.jpg', 'data/obj/data/images (71).jpg', 'data/obj/data/images (42).jpg', 'data/obj/data/images (33).jpg', 'data/obj/data/images - 2022-07-31T105630.436.jpg', 'data/obj/data/images - 2022-07-31T105630.001.jpg', 'data/obj/data/images (50).jpg', 'data/obj/data/maxresdefault.jpg', 'data/obj/data/images - 2022-07-31T105631.009.jpg', 'data/obj/data/images - 2022-07-31T105631.021.jpg', 'data/obj/data/images (32).jpg', 'd

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

### 6) Start the training

In [16]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show > /mydrive/yolov3/train.log

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 2.216019, iou_loss = 0.000000, total_loss = 2.216019 
 total_bbox = 12566, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.335029), count: 4, class_loss = 3.781574, iou_loss = 2.314061, total_loss = 6.095635 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 2.442101, iou_loss = 0.000000, total_loss = 2.442101 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 2.213346, iou_loss = 0.000000, total_loss = 2.213346 
 total_bbox = 12570, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.450980), count: 4, class_loss = 4.035195, iou_loss = 0.801997, total_loss = 4.837193 
v3 (mse